In [127]:
import pennylane as qml
import torch
from torch import nn
from kan import KAN
import pennylane as qml
from pennylane import numpy as np
from RNN_block import RNN_block


n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs1,inputs2, weights1,weights2):
    block = RNN_block(8)
    block.embedding(inputs1)
    block.ansatz(weights1)
    block.embedding(inputs2)
    block.ansatz(weights2)
    return [qml.expval(qml.PauliZ(i)) for i in range(8)]

def quantum_layer(inputs1,inputs2, weights1,weights2):
    return quantum_circuit(inputs1,inputs2, weights1,weights2)

n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)


def generate_tensor(seed, size):
    """
    주어진 시드와 크기에 맞게 torch.Tensor를 생성합니다.

    Args:
        seed (int): 시드 값
        size (tuple): 생성할 텐서의 크기

    Returns:
        torch.Tensor: 생성된 텐서
    """
    torch.manual_seed(seed)
    return torch.randn(size)




class RNN_layer(nn.Module):
    def __init__(self,input_size,output_size,num_layers):
        """_RNN layer 만든거_

        Args:
            input_size (_int_): _input feature의 개수_
            output_size (_int_): _output feature의 개수_
            num_layers (_int_): _필요한 RNN layer 수_
        """
        super(RNN_layer, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_layer = num_layers
        self.cls_layer = nn.Sequential(nn.Linear(8,16),nn.ReLU(),nn.Linear(16,1))
        ## QNE 수행할 Linear layer
        for i in range(num_layers):
            setattr(self,f'QNE_layer_{i}',KAN([input_size,2*input_size+1,input_size*2-1],grid=1))
        
        ## Ansatz parameter
        self.ansatz_params_1 = nn.Parameter(torch.rand([24],dtype = torch.float32),requires_grad=True)
        self.ansatz_params_2 = nn.Parameter(torch.rand([24],dtype = torch.float32),requires_grad=True)
        self.rnn_layer = quantum_layer

        
    def forward(self,inputs,return_hidden_list = False):
        """_summary_

        Args:
            inputs (_torch tensor_): _(batch,seq_len,feature_size)_
        """
        index = 0
        
        batch = inputs.shape[0]
        seq_len = inputs.shape[1]
        initial_t = generate_tensor(30,[inputs.shape[0],inputs.shape[2]]).float()
        inputs = inputs.permute(1, 0, 2)
        ## inputs  = (seq_len,batch,feature_size)
        input = getattr(self,f'QNE_layer_{index}')(inputs[0])
        index+=1
        hidden = torch.stack(self.rnn_layer(input,initial_t,self.ansatz_params_1,self.ansatz_params_2),dim=1).float()
        hidden = hidden.to(torch.float32)
        if return_hidden_list:
            hidden_list = hidden
        for input in inputs[1:]:
            input = getattr(self,f'QNE_layer_{index}')(input)
            index+=1
            hidden = torch.stack(self.rnn_layer(input,hidden,self.ansatz_params_1,self.ansatz_params_2),dim=1).float()
            hidden = hidden.to(torch.float32)
            if return_hidden_list:
                hidden_list = torch.concat([hidden_list,hidden])
        if return_hidden_list:
            hidden_list = torch.reshape(hidden_list,[batch,seq_len,-1])
            return hidden_list
        return self.cls_layer(hidden)


        
        
        
        
        
        
        
        
    



In [44]:
import pandas as pd

data_frame = pd.read_csv('./dataset.csv')

In [50]:
key_list = ['nat_demand','T2M_toc','QV2M_toc','TQL_toc','QV2M_san','W2M_san','QV2M_dav','W2M_dav']

In [51]:
feature1 = data_frame[key_list]

In [58]:
data_set  = torch.tensor(np.array(feature1))[:4200]
data_set = torch.reshape(data_set,[-1,7,8])

In [68]:
label_list = [7*i+8 for i in range(400) ]

In [82]:
train_data = data_set[:400]
train_label = torch.tensor(np.array(data_frame['nat_demand'].iloc[label_list]))
test_data = data_set[400:]

In [130]:
train_rnn_layer(train_data.float() ,train_label.float())

Epoch [1/500], Loss: 1263785.2500
Epoch [2/500], Loss: 1261866.0000
Epoch [3/500], Loss: 1259216.1250
Epoch [4/500], Loss: 1254899.3750
Epoch [5/500], Loss: 1249093.3750
Epoch [6/500], Loss: 1240819.0000
Epoch [7/500], Loss: 1230358.5000
Epoch [8/500], Loss: 1217994.7500
Epoch [9/500], Loss: 1196244.7500
Epoch [10/500], Loss: 1178676.2500
Epoch [11/500], Loss: 1145298.1250
Epoch [12/500], Loss: 1118004.1250
Epoch [13/500], Loss: 1087227.1250
Epoch [14/500], Loss: 1063077.8750
Epoch [15/500], Loss: 1026392.8750
Epoch [16/500], Loss: 982157.5000
Epoch [17/500], Loss: 939556.7500
Epoch [18/500], Loss: 897312.6875
Epoch [19/500], Loss: 845707.8125
Epoch [20/500], Loss: 722240.1250
Epoch [21/500], Loss: 557006.8750
Epoch [22/500], Loss: 388910.4688
Epoch [23/500], Loss: 322447.2188
Epoch [24/500], Loss: 248862.9219
Epoch [25/500], Loss: 188042.4375
Epoch [26/500], Loss: 140311.4844
Epoch [27/500], Loss: 134089.9375
Epoch [28/500], Loss: 117622.2734
Epoch [29/500], Loss: 76857.1172
Epoch [30

KeyboardInterrupt: 

In [131]:
import torch.optim as optim
# Define the training loop



def train_rnn_layer(inputs,targets,model):
    # Parameters
    input_size = 8
    output_size = 8
    num_layers = 7
    num_epochs = 500
    learning_rate = 0.2


    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Dummy data for training

    # Training loop
    for epoch in range(num_epochs):

        # Forward pass
        outputs = model(inputs)
        outputs = outputs.to(torch.float32)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [132]:
model = RNN_layer(8, 8, 2)

In [124]:
model.parameters

<bound method Module.parameters of RNN_layer(
  (cls_layer): Sequential(
    (0): Linear(in_features=8, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=1, bias=True)
  )
  (QNE_layer_0): KAN(
    (biases): ModuleList(
      (0): Linear(in_features=17, out_features=1, bias=False)
      (1): Linear(in_features=15, out_features=1, bias=False)
    )
    (act_fun): ModuleList(
      (0-1): 2 x KANLayer(
        (base_fun): SiLU()
      )
    )
    (base_fun): SiLU()
    (symbolic_fun): ModuleList(
      (0-1): 2 x Symbolic_KANLayer()
    )
  )
  (QNE_layer_1): KAN(
    (biases): ModuleList(
      (0): Linear(in_features=17, out_features=1, bias=False)
      (1): Linear(in_features=15, out_features=1, bias=False)
    )
    (act_fun): ModuleList(
      (0-1): 2 x KANLayer(
        (base_fun): SiLU()
      )
    )
    (base_fun): SiLU()
    (symbolic_fun): ModuleList(
      (0-1): 2 x Symbolic_KANLayer()
    )
  )
)>

In [122]:
model.ansatz_params_1

Parameter containing:
tensor([0.6167, 0.8152, 0.0657, 0.0846, 0.7246, 0.6801, 0.0346, 0.2212, 0.0425,
        0.1418, 0.2744, 0.9138, 0.4005, 0.7872, 0.9748, 0.7279, 0.0524, 0.7516,
        0.6907, 0.8875, 0.0068, 0.8075, 0.9093, 0.0348], requires_grad=True)

In [41]:
import torch
from torch import nn
import pennylane as qml
from pennylane import numpy as np

def test_rnn_layer():
    # Parameters for the RNN layer
    input_size = 8
    output_size = 8
    num_layers = 10

    # Initialize the RNN layer
    rnn_layer = RNN_layer(input_size, output_size, num_layers)

    # Generate dummy input data
    batch_size = 5
    seq_len = 10
    feature_size = input_size
    inputs = torch.randn(batch_size, seq_len, feature_size)

    # Forward pass through the RNN layer
    output = rnn_layer(inputs,return_hidden_list = True)

    # Print the output shape and the output itself
    print("Output shape:", output.shape)
    print("Output:", output)

test_rnn_layer()


Output shape: torch.Size([5, 10, 8])
Output: tensor([[[ 1.5881e-01, -3.2125e-01, -1.3448e-01, -3.6123e-01, -5.0410e-02,
          -1.7403e-02, -1.2152e-01,  4.4982e-02],
         [ 2.0169e-02,  6.4169e-01,  1.3500e-01,  3.3601e-01,  9.9510e-02,
           5.6186e-02,  8.2669e-02, -3.9853e-02],
         [ 5.6169e-01,  5.4541e-01,  2.0663e-01,  1.7761e-01, -6.0539e-02,
           6.4748e-02,  4.9791e-02, -6.1873e-02],
         [ 1.3180e-01, -2.9873e-01,  2.2480e-01,  3.9494e-01, -5.8878e-02,
           5.6205e-02,  1.7944e-01, -1.2226e-01],
         [ 9.4118e-02,  4.9910e-02,  3.6760e-01,  1.9947e-01,  9.7139e-02,
          -1.7423e-02,  5.7610e-02,  8.2776e-03],
         [ 3.3404e-02, -3.3880e-01, -1.9442e-01,  8.0139e-02,  1.5477e-02,
          -2.8924e-02,  3.4318e-02,  6.9795e-03],
         [ 7.9341e-01,  8.2112e-01,  2.8879e-01,  5.4817e-01, -2.3316e-01,
           1.3366e-01,  2.5388e-01, -2.2767e-01],
         [-7.6532e-02,  7.1550e-02,  2.8889e-01, -1.1167e-01, -5.4760e-02,
     

In [9]:
import pennylane as qml
class RNN_block:
    def __init__(self, input_size):
        self.input_size = input_size

    
    def embedding(self, params):
        n = (self.input_size + 1) // 2
        for i in range(n):
            qml.Hadamard(i)
            qml.RZ(2.0 * params[:,i], i)
        
        for i in range(n - 1):
            qml.IsingZZ(2.0 * params[:,n + i] ,[i , i + 1])
    
    def ansatz(self, params, all_entangled = False):
        # Length of Params : 3 * num_qubit
        n = self.input_size
        for i in range(n):
            qml.RX(params[3 * i], i)
            qml.RY(params[3 * i + 1], i)
            qml.RZ(params[3 * i + 2], i)
        for i in range(n - 1):
            qml.CNOT([i, i + 1])
        if all_entangled:
            qml.CNOT([n - 1, 0])

In [15]:
n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs1,inputs2, weights1,weights2):
    block = RNN_block(8)
    block.embedding(inputs1)
    block.ansatz(weights1)
    block.embedding(inputs2)
    block.ansatz(weights2)
    return [qml.expval(qml.PauliZ(i)) for i in range(8)]
def quantum_layer(inputs1,inputs2, weights1,weights2):
    return quantum_circuit(inputs1,inputs2, weights1,weights2)

torch.stack(quantum_circuit(torch.rand([4,15]),torch.rand([4,15]),torch.rand([24]),torch.rand([24])),dim=1).shape

torch.Size([4, 8])